In [4]:
import copy
import pandas as pd

# 读入 CSV，并把“启用时间”和“变更（弃用）时间”转成 Int64
df = pd.read_csv('../result.csv', dtype=str)
df['启用时间'] = pd.to_numeric(df['启用时间'], errors='coerce').astype('Int64')
df['变更（弃用）时间'] = pd.to_numeric(df['变更（弃用）时间'], errors='coerce').astype('Int64')

years = list(range(1980, 2025))

# 最终要生成的“宽表行”列表，以及 code→行号映射
result_rows = []
code_to_row = {}

# 用于存放每年“旧→新”的帮助表
lineage_dict = {}

# —— 1) 先处理 1980 年 —— 
for _, row in df[df['启用时间'] == 1980].iterrows():
    code0 = row['代码']
    entry = {
        f'代码_1980': code0,
        f'一级行政区_1980': row['一级行政区'],
        f'二级行政区_1980': row['二级行政区'],
        f'名称_1980': row['名称'],
        f'级别_1980': row['级别']
    }
    result_rows.append(entry)
    code_to_row[code0] = len(result_rows) - 1

# —— 2) 从 1981 年到 2024 年，逐年做“变更/弃用”→“拆分/更新”→“全新”三步 —— 
for y in years[1:]:
    # 2.1) 筛出当年“变更”或“弃用”的行，构造 lineage_{y}
    change_rows = df[(df['变更（弃用）时间'] == y) & (df['状态'].isin(['变更', '弃用']))]
    lineage_entries = []

    # 先把所有变更条目拆成多个“旧→新”，放到帮助表
    for _, crow in change_rows.iterrows():
        old_code = crow['代码']
        old_d1   = crow['一级行政区']
        old_d2   = crow['二级行政区']
        old_name = crow['名称']
        old_lvl  = crow['级别']
        raw_new  = str(crow['新代码']).split(';')

        valid_new = [nc for nc in raw_new if nc.isdigit() and len(nc) == 6]

        for nc in valid_new:
            lineage_entries.append({
                '旧代码': old_code,
                '旧一级行政区': old_d1,
                '旧二级行政区': old_d2,
                '旧名称': old_name,
                '旧级别': old_lvl,
                '新代码': nc
            })

    lineage_df = pd.DataFrame(lineage_entries, columns=[
        '旧代码', '旧一级行政区', '旧二级行政区', '旧名称', '旧级别', '新代码'
    ])
    lineage_dict[y] = lineage_df

    # 2.2) 在宽表中，对每一个“旧→新”条目执行以下操作：
    #      如果这条“旧”曾经存在于上一年（即 code_to_row 中有记录），
    #      并且 valid_new 不止一个，就把上一年(1980~y-1)的所有列复制给多个分支行；
    #      如果 valid_new 只有一个，就直接在原行填上 y 年的信息。

    # 先记录每个旧代码对应的所有新代码列表，便于一次性判断“分裂情况”
    old_to_all_new = {}
    for _, lee in lineage_df.iterrows():
        old_to_all_new.setdefault(lee['旧代码'], []).append(lee['新代码'])

    # 遍历 old_to_all_new，处理“分裂”：
    for old_code, new_list in old_to_all_new.items():
        if old_code not in code_to_row:
            # 如果旧代码在宽表中尚未出现，（理论上不应发生），就跳过
            continue

        old_row_idx = code_to_row[old_code]
        # 如果只是单一映射，后面 update 代码逻辑里会直接覆盖；但如果是分裂，需要先复制多行
        if len(new_list) > 1:
            # 先取出“复制前”的原行 dict（它包含 1980 ~ (y-1) 年的所有已填字段）
            base_dict = result_rows[old_row_idx]
            # 把原行从宽表中“删除”或标记为“到 y−1 年就结束”
            # 这里的做法：将原行从 result_rows 中 pop 掉，并且以后 code_to_row.pop(old_code)
            _ = result_rows.pop(old_row_idx)
            code_to_row.pop(old_code)

            # 因为我们 pop 掉一行，后面所有索引可能都往前挪 1，需要修正 code_to_row 里其他条目的行号
            for code_k, idx_v in list(code_to_row.items()):
                if idx_v > old_row_idx:
                    code_to_row[code_k] = idx_v - 1

            # 把那条旧行在“1980~(y−1)年”所有列，复制 len(new_list) 份
            for nc in new_list:
                new_dict = copy.deepcopy(base_dict)
                # 填写第 y 年的“代码、一级行政区、二级行政区、名称、级别”信息
                new_info = df[df['代码'] == nc].iloc[0]
                new_dict[f'代码_{y}'] = nc
                new_dict[f'一级行政区_{y}'] = new_info['一级行政区']
                new_dict[f'二级行政区_{y}'] = new_info['二级行政区']
                new_dict[f'名称_{y}'] = new_info['名称']
                new_dict[f'级别_{y}'] = new_info['级别']

                # 把这一行追加到 result_rows；记录行号给 mapping
                result_rows.append(new_dict)
                code_to_row[nc] = len(result_rows) - 1

        # 如果 new_list 只有一个，在下面的“单一映射” 更新逻辑里会被直接覆盖填写

    # 2.3) 继续处理单一映射：那些并未被当作“分裂”处理或本来只有一对一的“旧→新”
    for _, lee in lineage_df.iterrows():
        old_code = lee['旧代码']
        new_code = lee['新代码']

        # 若这对映射已经在上一步（len(new_list)>1 时）被处理（即旧行被 pop 了）
        # 那么此时 code_to_row[old_code] 会 KeyError，或者 new_code 已经被映射到一个新行
        if new_code in code_to_row and old_code not in code_to_row:
            # 说明这一对已经在“分裂”步骤里写入了 new_code → 目标行
            continue

        # 单一映射的情况：只要 old_code 仍存在于 code_to_row，按原逻辑更新旧行即可
        if old_code in code_to_row:
            row_idx = code_to_row[old_code]
            new_info = df[df['代码'] == new_code].iloc[0]
            result_rows[row_idx][f'代码_{y}']     = new_code
            result_rows[row_idx][f'一级行政区_{y}'] = new_info['一级行政区']
            result_rows[row_idx][f'二级行政区_{y}'] = new_info['二级行政区']
            result_rows[row_idx][f'名称_{y}']     = new_info['名称']
            result_rows[row_idx][f'级别_{y}']     = new_info['级别']
            # 更新 mapping
            code_to_row[new_code] = row_idx
            # 把旧的 mapping 删除（因为旧代码在这一年就废止或变更）
            code_to_row.pop(old_code, None)

    # 2.4) 处理“启用时间 = y”的全新编码：这些编码如果不在本年“旧→新”的 new_code 集合里，就是 brand-new
    df_enabled = df[df['启用时间'] == y]
    enabled_codes = set(df_enabled['代码'])
    new_codes_from_change = set(lineage_df['新代码'])
    brand_new = enabled_codes - new_codes_from_change

    for bn in brand_new:
        info_row = df[df['代码'] == bn].iloc[0]
        new_entry = {
            f'代码_{y}': bn,
            f'一级行政区_{y}': info_row['一级行政区'],
            f'二级行政区_{y}': info_row['二级行政区'],
            f'名称_{y}': info_row['名称'],
            f'级别_{y}': info_row['级别']
        }
        result_rows.append(new_entry)
        code_to_row[bn] = len(result_rows) - 1

# —— 3) 最后，把所有行合并成一个 DataFrame，所有年份对应的列都列出来，不存在的用 NaN 填充 —— 
all_cols = []
for yr in years:
    all_cols += [
        f'代码_{yr}',
        f'一级行政区_{yr}',
        f'二级行政区_{yr}',
        f'名称_{yr}',
        f'级别_{yr}'
    ]

result_df = pd.DataFrame(result_rows).reindex(columns=all_cols)

result_df.to_csv('../output_data/result_wide.csv', index=False)

GPT Prompt:

https://chatgpt.com/share/683f0d30-aaa0-800e-aeac-0a35cf91d5e7
